In [1]:
#try xgboost
import pandas as pd
import numpy as np
import xgboost as xgb

In [2]:
df = pd.read_csv('/Users/frankieshen/Desktop/ooschool/專案/stock_0050_final.csv', parse_dates = True, index_col = 0)

In [3]:
print(df.columns)
print(df.shape)
print(type(df))

Index(['Unnamed: 0', 'Capacity', 'Volume', 'Open', 'High', 'Low', 'Close',
       'Change', 'Transcation', 'Tomorrow', 'Target', 'SMA_20', 'RSI', 'slowk',
       'slowd', 'MACD', 'MACD_signal', 'MACD_hist', 'upper_band',
       'middle_band', 'lower_band'],
      dtype='object')
(3483, 21)
<class 'pandas.core.frame.DataFrame'>


In [4]:
df.head()

,Unnamed: 0,Capacity,Volume,Open,High,Low,Close,Change,Transcation,Tomorrow,...,SMA_20,RSI,slowk,slowd,MACD,MACD_signal,MACD_hist,upper_band,middle_band,lower_band
Date,,,,,,,,,,,,,,,,,,,,,
2010-02-26,33,9391284,483837797,51.65,51.75,51.20,51.55,-0.10,1366,52.80,...,52.4075,38.424149,46.288720,50.743915,-1.500417,-1.927544,0.427127,54.411814,52.4075,50.403186
2010-03-01,34,13647130,715419506,51.50,52.95,51.50,52.80,1.25,2066,52.65,...,52.3275,47.850461,58.861819,52.243188,-1.336577,-1.809351,0.472774,54.124296,52.3275,50.530704
2010-03-02,35,9997438,528112268,52.75,53.10,52.55,52.65,-0.15,1225,52.65,...,52.2500,46.922188,71.092962,58.747834,-1.204946,-1.688470,0.483524,53.838710,52.2500,50.661290
2010-03-03,36,8764746,462426932,52.70,53.10,52.50,52.65,0.00,1041,52.20,...,52.2350,46.922188,84.210526,71.388436,-1.088085,-1.568393,0.480308,53.802514,52.2350,50.667486
2010-03-04,37,8149453,426373974,52.75,52.90,51.80,52.20,-0.45,1295,52.95,...,52.2000,43.955363,77.192982,77.498824,-1.020025,-1.458719,0.438694,53.737530,52.2000,50.662470


In [5]:
df_reorder = df[['Volume','Open', 'High', 'Low', 
              'SMA_20', 'RSI', 'slowk', 'slowd', 'MACD', 'MACD_signal', 'MACD_hist', 'upper_band', 'middle_band', 'lower_band', 'Close']]

In [6]:
print(df_reorder.columns)
print(df_reorder.shape)
print(type(df_reorder))

Index(['Volume', 'Open', 'High', 'Low', 'SMA_20', 'RSI', 'slowk', 'slowd',
       'MACD', 'MACD_signal', 'MACD_hist', 'upper_band', 'middle_band',
       'lower_band', 'Close'],
      dtype='object')
(3483, 15)
<class 'pandas.core.frame.DataFrame'>


In [7]:

def windowing(train, val, WINDOW, PREDICTION_SCOPE):
    assert type(train) == np.ndarray, "train must be passed as an array"
    assert type(val) == np.ndarray, "validation must be passed as an array"
    assert type(WINDOW) == int, "Window must be an integer"
    assert type(PREDICTION_SCOPE) == int, "Prediction score must be an integer"

    x_train = []
    y_train = []
    x_test = []
    y_test = []

    for i in range(len(train) - (WINDOW + PREDICTION_SCOPE)):
        x, y = np.array(train[i:i + WINDOW, :-1]), np.array(train[i + WINDOW + PREDICTION_SCOPE, -1])
        x_train.append(x)
        y_train.append(y)
    for i in range(len(val) - (WINDOW + PREDICTION_SCOPE)):
        x, y = np.array(val[i:i + WINDOW, :-1]), np.array(val[i + WINDOW + PREDICTION_SCOPE, -1])
        x_test.append(x)
        y_test.append(y)


    return x_train, y_train, x_test, y_test

In [9]:
def train_test_split(data, percentage):
    assert type(data) == pd.core.frame.DataFrame, "data must be a dataframe"
    assert type(percentage) == float, "percentage must be a float"

    train = data.iloc[:int(len(data) * percentage)]
    test = data.iloc[int(len(data) * percentage):]
    
    return train, test

In [10]:
def train_validation_split(train, percentage):
    assert type(train) == pd.core.frame.DataFrame, "train musst be a dataframe"
    assert type(percentage) == float, "percentage musst be a float"
    
    train_set = np.array(train.iloc[:int(len(train) * percentage)])
    validation_set = np.array(train.iloc[int(len(train) * percentage):])
    
    return train_set, validation_set

In [11]:
percentage = 0.8
WINDOW = 5
PREDICTION_SCOPE = 0

In [13]:
train, test = train_test_split(df_reorder, percentage)
train_set, validation_set = train_validation_split(train, percentage)
print(f"train_set shape: {train_set.shape}")
print(f"validation_set shape: {validation_set.shape}")
print(f"test shape: {test.shape}")
print(f"train shape: {train.shape}")

train_set shape: (2228, 15)
validation_set shape: (558, 15)
test shape: (697, 15)
train shape: (2786, 15)


In [14]:
X_train, y_train, X_val, y_val = windowing(train_set, validation_set, WINDOW, PREDICTION_SCOPE)
#Convert the returned list into arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {y_val.shape}")

X_train shape: (2223, 5, 14)
y_train shape: (2223,)
X_val shape: (553, 5, 14)
y_val shape: (553,)


In [16]:
#xgboost only allows two dimensional input
#reshape x_train and x_val into two dimension by joining the rows of the windowed days into one big input
X_train_2dim = X_train.reshape(X_train.shape[0], -1)
X_val_2dim = X_val.reshape(X_val.shape[0], -1)
print(f"X_train_new shape: {X_train_2dim.shape}")
print(f"X_val_new shape: {X_val_2dim.shape}")

X_train_new shape: (2223, 70)
X_val_new shape: (553, 70)


In [17]:
xgb_model = xgb.XGBRegressor(gamma=1, n_estimators=200)
xgb_model.fit(X_train_2dim, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [18]:
from sklearn.metrics import mean_absolute_error

In [26]:
pred_val = xgb_model.predict(X_val_2dim)
mae_val = mean_absolute_error(y_val, pred_val)
mae_val

16.271520689814256

In [43]:
#test the model
X_test = np.array(test.iloc[:, :-1])
y_test = np.array(test.iloc[:, -1])
print(X_test.shape)
print(y_test.shape)

(697, 14)
(697,)


In [44]:
pred_test = xgb_model.predict(X_test)
#ValueError: Feature shape mismatch, expected: 70, got 14

ValueError: Feature shape mismatch, expected: 70, got 14